# 索引和形状表达式

你已经知道形状可以是符号元组，例如 `(n, m)`，元素可以通过索引访问，例如 `a[i, j]`。在实践中，形状和索引都可以通过复杂的表达式来计算。

In [1]:
# from tvm_book.contrib import d2ltvm
import numpy as np
import tvm
from tvm import te

## 转置(te)

第一个例子是矩阵转置 `a.T`，按列访问 `a` 中的元素。

In [2]:
n = te.var('n')
m = te.var('m')
A = te.placeholder((n, m), name='a')
B = te.compute((m, n), lambda i, j: A[j, i], 'b')
s = te.create_schedule(B.op)
ir_mod = tvm.lower(s, [A, B], simple_mode=True)
ir_mod['main']

PrimFunc([a, b]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  for (i, 0, m) {
    for (j, 0, n) {
      b[((i*stride) + (j*stride))] = a[((j*stride) + (i*stride))]
    }
  }
}

注意 2-D 索引，例如 `b[i,j]`  按照 C 约定折叠到（collapsed） 1-D 索引 `b[((i*n) + j)]`。

验证结果。

In [3]:
a = np.arange(12, dtype='float32').reshape((3, 4))
b = np.empty((4, 3), dtype='float32')
a, b = tvm.nd.array(a), tvm.nd.array(b)

mod = tvm.build(ir_mod)
mod(a, b)
a, b

(<tvm.nd.NDArray shape=(3, 4), cpu(0)>
 array([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]], dtype=float32),
 <tvm.nd.NDArray shape=(4, 3), cpu(0)>
 array([[ 0.,  4.,  8.],
        [ 1.,  5.,  9.],
        [ 2.,  6., 10.],
        [ 3.,  7., 11.]], dtype=float32))

## `reshape`

接下来使用表达式进行索引。下面的代码块将2-D 数组 `a`（上面定义的 $n \times m$）重构为 1-D（就像 NumPy 中的`a.reshape(-1)`）。注意如何将 1-D 索引 `i` 转换为 2-D 索引 `[i//m, i%m]`。

In [4]:
B = te.compute((m*n, ), lambda i: A[i//m, i%m], name='b')
s = te.create_schedule(B.op)
ir_mod = tvm.lower(s, [A, B], simple_mode=True)
ir_mod['main']

PrimFunc([a, b]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  for (i, 0, (m*n)) {
    b[i] = a[((floordiv(i, m)*stride) + (floormod(i, m)*stride))]
  }
}

由于 $n$-D 数组在内存中实际上是作为 1-D 数组列出的，因此生成的代码并不重新排列数据序列，而是将索引表达式从 2-D (`(i//m)*m + i%m`) 简化为 1-D (`i`)，以提高效率。

也可以实现一般的二维重构函数。

In [5]:
p, q = te.var('p'), te.var('q')
B = te.compute((p, q), lambda i, j: A[(i*q+j)//m, (i*q+j)%m], name='b')
s = te.create_schedule(B.op)
ir_mod = tvm.lower(s, [A, B], simple_mode=True)
ir_mod["main"]

PrimFunc([a, b]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  for (i, 0, p) {
    for (j, 0, q) {
      b[((i*stride) + (j*stride))] = a[((floordiv(((i*q) + j), m)*stride) + (floormod(((i*q) + j), m)*stride))]
    }
  }
}

在测试结果时，应该意识到，没有对输出形状施加约束，它可以有任意形状 `(p, q)`，因此 TVM 将无法检查 $qp = nm$。例如，在下面的例子中，创建了 `b`，其尺寸 (20) 比 `a` (12) 大，那么 `b` 中只有前 12 个元素来自 `a` ，其他的都是未初始化的值。

In [6]:
mod = tvm.build(ir_mod)
a = np.arange(12, dtype='float32').reshape((3, 4))
b = np.zeros((5, 4), dtype='float32')
a, b = tvm.nd.array(a), tvm.nd.array(b)

mod(a, b)
print(b)

[[ 0.0000000e+00  1.0000000e+00  2.0000000e+00  3.0000000e+00]
 [ 4.0000000e+00  5.0000000e+00  6.0000000e+00  7.0000000e+00]
 [ 8.0000000e+00  9.0000000e+00  1.0000000e+01  1.1000000e+01]
 [-2.7943063e-24  3.0936466e-41  1.5834673e-43  0.0000000e+00]
 [-2.4915414e-24  3.0936466e-41 -2.4883166e-24  3.0936466e-41]]


## 张量切片（te）

考虑特殊的切片算子 `a[bi::si, bj::sj]`，其中 `bi`，`bj`，`si` 和 `sj` 可以稍后指定。现在需要根据参数计算输出形状。此外，需要在编译模块时将变量 `bi`，`bj`，`si` 和 `sj` 作为参数传递。

In [7]:
bi, bj, si, sj = [te.var(name) for name in ['bi', 'bj', 'si', 'sj']]
B = te.compute(((n-bi)//si, (m-bj)//sj),
               lambda i, j: A[i*si+bi, j*sj+bj],
               name='b')
s = te.create_schedule(B.op)
mod = tvm.build(s, [A, B, bi, si, bj, sj])

现在测试两个案例来验证正确性。

In [8]:
b = tvm.nd.array(np.empty((1, 3), dtype='float32'))
mod(a, b, 1, 2, 1, 1)
np.testing.assert_equal(b.numpy(), a.numpy()[1::2, 1::1])

b = tvm.nd.array(np.empty((1, 2), dtype='float32'))
mod(a, b, 2, 1, 0, 2)
np.testing.assert_equal(b.numpy(), a.numpy()[2::1, 0::2])

## 小结

- 形状维度和索引都可以是带有变量的表达式。
- 如果变量不仅出现在形状元组中，那么需要在编译时将其作为参数传递。